In [4]:
%pwd
import os
os.chdir('..')

In [5]:
%pwd

'd:\\Code\\mlops\\10.endtoend\\endtoend'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionCofig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [7]:
from src.datasci.constants import (CONFIG_FILE_PATH, PARAMS_FILE_PATH,
                                   SCHEMA_FILE_PATH)
from src.datasci.utils.common import create_directories, read_yaml
from box import ConfigBox


class ConfiguratonManager:
    def __init__(self,
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH,
                 schema_file_path=SCHEMA_FILE_PATH
                 ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self)-> DataIngestionCofig:
        config = self.config.data_ingestion #from yaml
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionCofig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [8]:
import os
import urllib.request as req
from src.datasci import logger
import zipfile

#component
class DataIngestion:
    def __init__(self,config:DataIngestionCofig) -> None:
        self.config = config
    
    #download the file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename,headers =  req.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
                logger.info(f"file already exists of size")
           
    def extract_zip_file(self):
        """
        
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [9]:
try:
    config = ConfiguratonManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-12-06 15:10:31,068: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-06 15:10:31,074: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-06 15:10:31,079: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-06 15:10:31,083: INFO: common: created directory at: artifacts]
[2024-12-06 15:10:31,087: INFO: common: created directory at: artifacts/data_ingestion]
[2024-12-06 15:10:33,165: INFO: 953279585: artifacts/data_ingestion/data.zip download! with following info: 
Server: GitHub.com
Date: Fri, 06 Dec 2024 09:40:31 GMT
Content-Type: text/html; charset=utf-8
Vary: X-PJAX, X-PJAX-Container, Turbo-Visit, Turbo-Frame, Accept-Encoding, Accept, X-Requested-With
ETag: W/"e89c4dd16e37ebd10f3bc9bba7fb8899"
Cache-Control: max-age=0, private, must-revalidate
Strict-Transport-Security: max-age=31536000; includeSubdomains; preload
X-Frame-Options: deny
X-Content-Type-Options: nosniff
X-XSS-Protection: 0
Referrer-Policy: no-referrer-when-dow

BadZipFile: File is not a zip file